# Objectives

In this lab you will

* Practice working with the linked-list polygon data structure. 
* Implement several basic convex hull algorithms. 

# Starter Code 

## Geometric Starter Code:

In [2]:
from koebe.geometries.euclidean2 import *
from random import random

def areaOfTriangle(A:PointE2, B:PointE2, C:PointE2):
    """Signed area of a triangle"""
    return 0.5 * ((B.x - A.x) * (C.y - A.y) - (C.x - A.x) * (B.y - A.y))

def leftHandTurn(A:PointE2, B:PointE2, C:PointE2):
    """Returns true iff. the points A, B, C constitute a left hand turn."""
    return areaOfParallelogram(A, B, C) > 0

def perturb(pointSet, scale=1e-8):
    """Randomly perturbs a list of points by a small amount."""
    eps = [((random()-0.5)*scale, (random()-0.5)*scale) for _ in range(len(pointSet))]
    return [PointE2(pointSet[i].x + eps[i][0], pointSet[i].y + eps[i][1]) for i in range(len(pointSet))]

class Vertex:
    """
    A class for a Vertex in a Polygon. Each vertex stores a PointE2 .pos for its
    coordinates, as well as .next and .prev attributes for the next and previous 
    vertices along the polygon.
    """
    def __init__(self, pos: PointE2):
        """
        Initializes the vertex at the given position.
        
        Attributes:
            pos: PointE2 - The position to initialize this vertex to.
        """
        self.pos = pos
        self.prev = None
        self.next = None
    
    def setNextVertex(self, nextVertex: "Vertex"):
        """
        Makes nextVertex the next vertex from this one and makes this vertex
        the previous vertex to nextVertex. 
        """
        self.next = nextVertex
        nextVertex.prev = self

class Polygon:
    """
    Simple container class for a Polygon. Should store a firstVertex. 
    As a convenience a list of PointE2 objects may be passed in to the 
    vertexPositions parameter, and the constructor will initialize a 
    doubly-connected linked list of Vertex objects with the PointE2s as the
    .pos attributes. 
    """
    def __init__(self, vertexPositions = []):
        if len(vertexPositions) > 0:
            vertices = [Vertex(p) for p in vertexPositions]
            for i in range(len(vertices)):
                vertices[i - 1].setNextVertex(vertices[i])
            self.firstVertex = vertices[0]
        else:
            self.firstVertex = None
    
    def __iter__(self):
        """
        An iterator for the polygon that allows you to loop
        over a polygon's vertices using: 
        
        for vertex in polygon:
            ... do something with the vertex object ...
        """
        current = self.firstVertex
        while True:
            yield current
            current = current.next
            if current is self.firstVertex:
                break
    
    def vertexPositions(self):
        """
        Returns the positions of all ther vertices in order.
        """
        return [v.pos for v in self] # Uses the __iter__ to build a list
    
    def copy(self):
        """
        Returns a copy of this polygon. This performs a shallow copy
        since PointE2 objects are immutable. 
        """
        return Polygon(self.vertexPositions())

## Graphics Starter Code: 

Here we package up the `ipycanvas` drawing code into a convenience function called `PointEditorCanvas` that takes three parameters:

* `size`, a tuple for the size of the canvas (e.g. `(600, 600)`). 
* `points`, the initial point set given as a list of `PointE2` objects. 
* `draw_func`, a function that takes a canvas and the current point set as its two parameters. 

By the way, this function is using some nifty higher-order functional programming with closures. If you're interested, ask me about it. 

In [3]:
from ipycanvas import Canvas, hold_canvas

def PointEditorCanvas(size, pts, draw_func = None):
    
    points = list(pts)
    canvas = Canvas(size=size)
    selectedPointIdx = -1
    
    def _canvas_draw():
        nonlocal canvas, points, selectedPointIdx, draw_func
        
        with hold_canvas(canvas):
            canvas.clear()

            if draw_func != None:
                draw_func(canvas, points)

            canvas.fill_style = 'blue'
            canvas.fill_rects([round(p.x) - 4 for p in points], 
                              [round(p.y) - 4 for p in points], 
                              8)
            if selectedPointIdx != -1:
                canvas.fill_style = 'red'
                canvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                                 round(points[selectedPointIdx].y) - 4, 
                                 8)
            
    def handle_mouse_down(x, y):
        nonlocal selectedPointIdx
        # See if any point is close to x, y
        cursorPoint = PointE2(x, y)
        sqDists = [p.distSqTo(cursorPoint) for p in points]
        minIdx = sqDists.index(min(sqDists))
        if sqDists[minIdx] < 24:
            selectedPointIdx = minIdx
        _canvas_draw()

    def handle_mouse_up(x, y):
        nonlocal selectedPointIdx
        selectedPointIdx = -1 # No point is selected anymore.
        _canvas_draw()

    def handle_mouse_move(x, y):
        nonlocal selectedPointIdx
        if selectedPointIdx >= 0:
            points[selectedPointIdx] = PointE2(x, y)
        _canvas_draw()
    
    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_up(handle_mouse_up)
    canvas.on_mouse_move(handle_mouse_move)
    
    _canvas_draw()
    
    return canvas

Here's an example of how to use the `PointEditorCanvas` to simply draw a polygon on the point set. 

In [5]:
def drawPolygon(canvas, points):
    
    polygon = Polygon(perturb(points))

    canvas.begin_path()
    canvas.move_to(round(polygon.firstVertex.pos.x), round(polygon.firstVertex.pos.y))
    for p in polygon.vertexPositions():
        canvas.line_to(round(p.x), round(p.y))
    canvas.close_path()
    canvas.stroke()


PointEditorCanvas(
    size=(600,600), 
    pts=[PointE2(100, 200), PointE2(200, 200), 
         PointE2(200, 300), PointE2(200, 400), 
         PointE2(300, 200), PointE2(300, 300), 
         PointE2(300, 400), PointE2(400, 400), 
         PointE2(500, 100)], 
    draw_func=drawPolygon
)

Canvas(layout=Layout(height='600px', width='600px'), size=(600, 600))

In [31]:
def orient(tri):
    """
    A helpful convenience method that takes a list tri of three PointE2 objects
    and returns the list in ccw order. You may find this useful in orienting 
    your polygon. 
    """
    if leftHandTurn(*tri):
        return tri
    else:
        return list(reversed(tri))

def findLowerTangent(poly, p):
    """
    Returns the vertex that is a lower tangent for the polygon with p. 
    """
    pass # TODO

def findUpperTangent(poly, p):
    """
    Returns the vertex that is an upper tangent for the polygon with p.
    """
    pass # TODO

def incrConv(P):
    
    """
    Computes the convex hull ofa list P of PointE2s using the incremental algorithm. 
    
    Assumes P has at least three points in it. 
    """
    thePoints = list(P) # Make a copy of P. 
    
    # TODO

# Task 1: 

Complete the code above (fill in for the `# TODO` markers) to implement the incremental algorithm for computing convex hulls. Then, in the code block below define a `drawIncrConvHull` function modeled on the `drawPolygon` function above and use it to draw a `PointEditorCanvas` that maintains the convex hull in the code block below:

In [7]:
# Your drawing code here

# Task 2: 

You will now implement the $O(n\log n)$ Graham scan algorithm for computing convex hulls. The pseudo code is given in the comments. 

## Sorting in Python

Before we let's look at how to do sorting in python. 

__WARNING:__ This is a little bit advanced from a programming perspective. If this goes over your head right now, don't worry about it, I'm happy to help you get these methods implemented. 

In [15]:
aList = [2, 3, 1, 5, 7, 8, 1, 5, 2]

# We can sort a list using the element's built in comparison function:

sList = sorted(aList)
print(f"Sorted: {sList}")

Sorted: [1, 1, 2, 2, 3, 5, 5, 7, 8]
Sorted: [8, 7, 5, 5, 3, 2, 2, 1, 1]
Sorted [PointE2(x=1, y=5), PointE2(x=3, y=1), PointE2(x=3, y=4), PointE2(x=3, y=7), PointE2(x=10, y=11)]
Sorted by distance to (3.2, 3.4) [PointE2(x=3, y=4), PointE2(x=3, y=1), PointE2(x=1, y=5), PointE2(x=3, y=7), PointE2(x=10, y=11)]


In [ ]:

# We can optionally supply a "key" function, that creates the value to be used sorting
# For example, if we want to sort in reverse order, we could flip the sign of each 
# number to be its key. 

def negate(x):
    return -x

sList = sorted(aList, key=negate)
print(f"Sorted: {sList}")


In [ ]:

# It is a little trickier to create a comparator function that compares two values. 
# This is typically done by providing a function comp(x, y) that returns -1 if x
# comes before y in sorted order, 0 if they should be considered the same, or 
# 1 if x comes after y. For example: 

def comparePoints(p, q):
    """
    Compares points first by x-coordinate and then breaks ties by y-coordinate. 
    """
    if p.x < q.x:
        return -1
    elif p.x > q.x:
        return 1
    elif p.y < q.y:
        return -1
    elif p.y > q.y:
        return 1
    else:
        return 0
    
pointList = [PointE2(3, 4), PointE2(3, 1), PointE2(3, 7), PointE2(10, 11), PointE2(1, 5)]


In [ ]:

# To use a comparator function like comparePoints we have to convert it using
# some python magic to a key function using a library called func_tools

from functools import cmp_to_key

sList = sorted(pointList, key=cmp_to_key(comparePoints))
print(f"Sorted {sList}")


In [ ]:

# Now let's do something super tricky. We would like to sort points by the distance 
# to some other point, say: 
specialPoint = PointE2(3.2, 3.4)

def distCompare(p, q):
    global specialPoint
    
    dp = specialPoint.distSqTo(p)
    dq = specialPoint.distSqTo(q)
    
    if dp < dq:
        return -1
    elif dp > dq:
        return 1
    else:
        return 0

sList = sorted(pointList, key=cmp_to_key(distCompare))
print(f"Sorted by distance to (3.2, 3.4) {sList}")


In [25]:
# The problem with this approach is the use of a global variable specialPoint. What
# if I want to have different versions of the distCompare function, each with a 
# different special point? We can use higher-order functions to achieve this:

# Let's first change distCompare so that it takes in the special point as a
# parameter:
def distCompare(specialPoint, p, q):
    dp = specialPoint.distSqTo(p)
    dq = specialPoint.distSqTo(q)
    
    if dp < dq:
        return -1
    elif dp > dq:
        return 1
    else:
        return 0

# Now we create a factory function that takes in a special point
# and partially applies the distCompare function to it. 
def distCompare_factory(specialPoint):
    from functools import cmp_to_key
    def distCompareWithSpecialPoint(p, q):
        return distCompare(specialPoint, p, q)
    return cmp_to_key(distCompareWithSpecialPoint)

sList = sorted(pointList, key=distCompare_factory(specialPoint))
print(f"Sorted by distance to (1, 1) {sList}")

Sorted by distance to (1, 1) [PointE2(x=3, y=4), PointE2(x=3, y=1), PointE2(x=1, y=5), PointE2(x=3, y=7), PointE2(x=10, y=11)]


In [29]:
# Notice that the nested function `distCompareWithSpecialPoint` 
# in `distCompare_factory` above is essentially the `distCompare` 
# function with its first parameter hard-coded to the specialPoint.
#
# This is called a partial application of the function, since we've 
# only passed the function some of the parameters, leaving the others free.
#
# Python's functools package has a shorthand for this called partial. 
# Here's a quick example: 

def add(x, y):
    print(f"x is {x}")
    print(f"y is {y}")
    return x + y

from functools import partial

add3to = partial(add, 3) # Create a new function, which is essentially add(3, y)
add3to(5) # We already gave add3to a valuef or x, we only need to give it y. 

x is 3
y is 5


8

In [30]:
# We can use this to simplify our distCompare_factory method: 

def distCompare_factory(specialPoint):
    from functools import partial, cmp_to_key
    return cmp_to_key(partial(distCompare, specialPoint))

sList = sorted(pointList, key=distCompare_factory(specialPoint))
print(f"Sorted by distance to (1, 1) {sList}")

Sorted by distance to (1, 1) [PointE2(x=3, y=4), PointE2(x=3, y=1), PointE2(x=1, y=5), PointE2(x=3, y=7), PointE2(x=10, y=11)]


## Your turn:

In [ ]:
def lht_compare(p, q0, q1):
    """
    Returns: 
         1 if p q0 q1 is a left hand turn, 
        -1 if p q0 q1 is a right hand turn, 
        if p q0 q1 are collinear (areaOfTriangle(p, q0, q1) == 0) then
            -1 if p is closer to q0 then q1
             1 if p is closer to q1 then q0
             0 otherwise
    """
    pass # TODO

def lht_comparator_factory(p):
    """
    This function uses partial function application and closures to 
    wrap the lht_compare function and close it with a given value for
    the first point p. 
    
    This can be used to sort a list of points by lht_compare using the 
    following:
    
    # Sorts a list of points pointList in ccw order around the 
    # given point p. 
    sortedPoints = sorted(pointList, key=lht_comparator_factory(p))
    """
    from functools import partial, cmp_to_key
    compare = partial(lht_compare, p)
    return cmp_to_key(compare)

def grahamScan_sortHelper(P):
    """
    Helper method that takes in a list of points P, finds the lowest point
    p by y coordinate, breaking ties by x coordinate and returns the list
    of points in order p in index 0 followed by the remaining points sorted
    using the lht_comparator_key. 
    """
    pass # TODO

def grahamScan(P):
    """
    Return the convex hull of P computed using the grahamScan method. 
    """
    pass # TODO

Once you've implemented the functions above in the code block below define a `drawGrahamScanConvHull` function modeled on the `drawPolygon` function above and use it to draw a `PointEditorCanvas` that maintains the convex hull in the code block below:

In [ ]:
# Your drawing code here.